# Ollama: Implementación de IA Local en Sistemas Agropecuarios

En esta clase exploraremos **Ollama**, una plataforma que permite la implementación de modelos de lenguaje grandes (LLM) en infraestructura local, complementando las soluciones cloud que hemos estudiado anteriormente.

## Ventajas estratégicas para el sector agropecuario

El sector agropecuario argentino, caracterizado por su alto nivel de tecnificación y digitalización, puede beneficiarse de implementaciones locales de IA por las siguientes razones:

- **Soberanía de datos**: Control total sobre información sensible (rendimientos, costos, estrategias comerciales)
- **Reducción de latencia**: Procesamiento inmediato sin dependencia de conectividad externa
- **Escalabilidad de costos**: Modelo económico predictible independiente del volumen de consultas
- **Integración con sistemas existentes**: Implementación en infraestructura de TI agropecuaria local

## Objetivos de aprendizaje

1. **Implementación básica**: Configuración y primera interacción con Ollama
2. **Customización de comportamiento**: Configuración de system prompts para contexto agronómico
3. **Extracción de datos estructurados**: Procesamiento de información agropecuaria en formatos estandarizados
4. **Procesamiento de archivos**: Análisis automatizado de reportes técnicos y datos de campo
5. **Desarrollo de interfaces**: Creación de herramientas de procesamiento de datos agropecuarios

## Fundamentos técnicos

Este cuaderno asume conocimientos básicos de Python e introduce gradualmente conceptos específicos de implementación de LLM locales. Los ejemplos están contextualizados en aplicaciones reales del sector agropecuario argentino.

**Nota metodológica**: Las celdas con llamadas a modelos pueden requerir tiempo de procesamiento considerable. En el contexto de clase, se pueden utilizar respuestas pregrabadas para optimizar el tiempo de instrucción.

---

## Configuración del entorno de desarrollo

### Instalación de dependencias

Comenzamos instalando las librerías necesarias para la implementación. Ollama requiere     
Python 3.8+ y las siguientes dependencias:
```bash
pip install ollama pandas gradio
```

### Importación de módulos y configuración inicial

El siguiente bloque importa las funciones necesarias y establece la configuración base para nuestro entorno de trabajo:

In [ ]:
# Importación de librerías core
import json  # Procesamiento de datos estructurados
import os    # Operaciones del sistema de archivos
import pandas as pd  # Análisis y manipulación de datos tabulares
from ollama import chat  # Cliente para interacción con modelos Ollama

# Configuración del modelo base
# Utilizaremos Gemma, optimizado para tareas de procesamiento de lenguaje natural
MODEL_NAME = "hf.co/unsloth/gemma-3n-E4B-it-GGUF:UD-Q4_K_XL"

print("Configuración completada. Entorno listo para implementación de IA local.")

## Implementación básica: Primera consulta agronómica

### Ejemplo: Consulta sobre densidad de siembra

Implementamos una consulta técnica sobre densidad de siembra de soja, un parámetro crítico en la producción agropecuaria argentina:

In [ ]:
# Definición de la consulta técnica
consulta_tecnica = """ ¿Cuáles son las densidades de siembra recomendadas para soja en la región pampeana argentina?
       Incluir variaciones por zona geográfica y condiciones de suelo.
       """
# Estructura del mensaje para el modelo
mensaje_consulta = [
       {
              "role": "user",
              "content": consulta_tecnica
              }
              ]

print("Ejecutando consulta al modelo local...")
print(f"Consulta: {consulta_tecnica}")

# Llamada al modelo Ollama

response = chat(
       model=MODEL_NAME,
       messages=mensaje_consulta
       )

# Extracción y presentación de la respuesta
respuesta_modelo = response.message.content
print("\n" + "="*80)
print("RESPUESTA DEL MODELO:")
print("="*80)
print(respuesta_modelo)

## System role: qué es y cómo definirlo

El *system role* (mensaje con `role="system"`) establece el comportamiento global del asistente: tono, estilo, límites, idioma y reglas generales. Se envía al inicio de la conversación y actúa como contexto persistente. Es útil para garantizar consistencia, reducir "hallucinations" y establecer reglas (por ejemplo: "no dar consejos médicos").

A continuación un ejemplo que compara una llamada sin system role y otra con system role.

In [ ]:
# System Prompt Especializado para Consultoría Agronómica

# Definimos el contexto profesional para el asistente de IA
system_prompt_agronomico = """
Eres un consultor agronómico con 20 años de experiencia especializado en la región pampeana argentina.
Tu expertise abarca:
- Cultivos extensivos (soja, maíz, trigo, girasol)
- Manejo integrado de plagas y enfermedades
- Nutrición vegetal y fertilización variable
- Tecnologías de precisión y agricultura digital
- Condiciones edafoclimáticas regionales
- Mercados agropecuarios y rentabilidad

Características de tus respuestas:
- Utiliza terminología técnica apropiada
- Incluye consideraciones económicas cuando sea relevante
- Referencias específicas a condiciones argentinas (suelos, clima, cultivares)
- Menciona tecnologías disponibles localmente
- Proporciona rangos de valores cuantitativos cuando sea apropiado

Si no tienes información específica sobre un tema, indica claramente esta limitación.
Responde siempre en español técnico profesional.
"""

# Consulta técnica sobre densidades de siembra
consulta_densidades = """
¿Cuáles son las densidades de siembra recomendadas para soja en la región pampeana argentina?
Incluir variaciones por zona geográfica, grupo de madurez y condiciones de suelo.
Considerar también el impacto económico de diferentes densidades.
"""

# Estructura del mensaje con system prompt
mensajes_consulta = [
    {"role": "system", "content": system_prompt_agronomico},
    {"role": "user", "content": consulta_densidades}
]

print("Ejecutando consulta con system prompt especializado...")
print(f"Modelo: {MODEL_NAME}")
print(f"Consulta: {consulta_densidades}")

# Llamada al modelo con contexto agronómico
response = chat(
    model=MODEL_NAME,
    messages=mensajes_consulta
)

# Extracción y presentación de la respuesta
respuesta_especializada = response.message.content
print("\n" + "="*80)
print("RESPUESTA DEL CONSULTOR AGRONÓMICO:")
print("="*80)
print(respuesta_especializada)

## Forzar un formato de salida (JSON)

Pedir al modelo que devuelva SOLO JSON facilita el parseo automático. Siempre indicar explícitamente: "Devuelve SOLO JSON, sin explicaciones adicionales". Aun así, el modelo puede añadir texto; por eso incluimos una heurística para extraer el JSON.

En entornos de producción podés validar la estructura con `jsonschema` o `pydantic`.

In [ ]:
# Extracción de Datos Estructurados: Análisis de Cosecha

def extract_json_from_text(text: str):
    """Extrae JSON válido de texto que puede contener explicaciones adicionales."""
    start = text.find('{')
    end = text.rfind('}')
    if start != -1 and end != -1 and end > start:
        candidate = text[start:end+1]
        try:
            return json.loads(candidate)
        except Exception:
            return None
    return None

# System prompt para análisis de datos agropecuarios
system_prompt_datos = """
Eres un analista de datos agropecuarios especializado en procesamiento de información de cosecha.
Extraes datos estructurados de reportes de campo para análisis posteriores.
Devuelve información en formato JSON válido con precisión técnica.
Incluye todos los parámetros productivos y económicos disponibles.
"""

# Reporte de cosecha real del campo La Esperanza
reporte_cosecha = """
REPORTE DE COSECHA 2024 - CAMPO LA ESPERANZA
Ubicación: Pergamino, Buenos Aires
Lote: 15-A
Superficie: 450 hectáreas
Cultivo: Soja DM4670 INTACTA RR2 PRO
Fecha de cosecha: 15/03/2024
Rendimiento promedio: 3,850 kg/ha
Humedad promedio: 12.8%
Proteína: 38.2%
Aceite: 19.1%
Peso de 1000 granos: 148.2 g
Precio estimado FOB: USD 425/tn
Costo directo total: USD 580/ha
Margen bruto estimado: USD 1,060/ha
Observaciones técnicas: Excelente llenado de granos pese a estrés hídrico en R3-R4.
Aplicación de fungicida en R5.1 con excelente timing.
"""

# Prompt para extracción estructurada
prompt_extraccion = f"""
Analiza el siguiente reporte de cosecha y extrae SOLO un JSON con la siguiente estructura:
{{
    "campo": "string",
    "ubicacion": "string",
    "lote": "string",
    "superficie_ha": float,
    "cultivo": "string",
    "fecha_cosecha": "string",
    "rendimiento_kg_ha": float,
    "humedad_pct": float,
    "proteina_pct": float,
    "aceite_pct": float,
    "peso_1000_granos_g": float,
    "precio_usd_tn": float,
    "costo_directo_usd_ha": float,
    "margen_bruto_usd_ha": float,
    "observaciones": "string"
}}

REPORTE A ANALIZAR:
{reporte_cosecha}

Devuelve SOLO el JSON, sin explicaciones adicionales.
"""

# Configurar mensajes
mensajes_extraccion = [
    {"role": "system", "content": system_prompt_datos},
    {"role": "user", "content": prompt_extraccion}
]

print("Procesando reporte de cosecha...")
print("Extrayendo datos estructurados con IA local...")

# Llamada al modelo
response = chat(model=MODEL_NAME, messages=mensajes_extraccion)

try:
    texto_respuesta = response.message.content
except Exception:
    texto_respuesta = str(response)

print('Respuesta del modelo:')
print(texto_respuesta)

# Extraer y procesar JSON
datos_estructurados = extract_json_from_text(texto_respuesta)
if datos_estructurados is None:
    print('\nNo se pudo extraer JSON válido del reporte.')
else:
    print('\n' + "="*60)
    print('DATOS DE COSECHA ESTRUCTURADOS:')
    print("="*60)
    print(json.dumps(datos_estructurados, indent=2, ensure_ascii=False))

    # Convertir a DataFrame para análisis
    df_cosecha = pd.DataFrame([datos_estructurados])
    print('\nDataFrame generado:')
    print(df_cosecha.to_string(index=False))

    # Cálculos adicionales de análisis económico
    produccion_total = datos_estructurados['superficie_ha'] * datos_estructurados['rendimiento_kg_ha'] / 1000
    ingreso_bruto_total = produccion_total * datos_estructurados['precio_usd_tn']
    margen_total = datos_estructurados['superficie_ha'] * datos_estructurados['margen_bruto_usd_ha']

    print(f'\n--- ANÁLISIS ECONÓMICO CONSOLIDADO ---')
    print(f'Producción total: {produccion_total:.1f} toneladas')
    print(f'Ingreso bruto total: USD {ingreso_bruto_total:,.0f}')
    print(f'Margen bruto total: USD {margen_total:,.0f}')

## Pasar un archivo .txt y devolver un pandas.DataFrame

Flujo recomendado:
1. Leer el archivo localmente en Python (si es muy grande, enviar un extracto o resumirlo).
2. Pedir al modelo que devuelva un JSON array donde cada elemento sea un objeto con las columnas deseadas.
3. Parsear el JSON en Python y construir un `pandas.DataFrame`.

A continuación hay un ejemplo completo. Cambiá `FILEPATH` por el archivo que quieras usar. Si el archivo no existe, se crea un ejemplo pequeño para que la celda sea reproducible.

In [ ]:
# Procesamiento de Reportes Fitosanitarios: Monitoreo de Plagas

# Creamos un reporte fitosanitario de ejemplo con datos técnicos reales
reporte_fitosanitario = """
REPORTE FITOSANITARIO SEMANAL - CAMPO SAN MARTÍN
Fecha: 15/01/2024
Responsable técnico: Ing. Agr. María López
Región: Norte de Buenos Aires

LOTE 12-B - SOJA DM4210 (V6-R1)
Superficie: 380 hectáreas
Fecha de siembra: 15/11/2023

MONITOREO DE PLAGAS:
1. CHINCHE VERDE (Nezara viridula):
   - Densidad promedio: 3.2 chinches/m lineal
   - Umbral de daño económico: 2.0 chinches/m lineal
   - Distribución: Focos en cabeceras, 15% superficie afectada
   - Recomendación: Aplicación inmediata requerida

2. ORUGA BOLILLERA (Helicoverpa gelotopoeon):
   - Densidad: 0.8 orugas/m lineal
   - Umbral: 1.5 orugas/m lineal
   - Estado: Bajo control, monitoreo continuado

LOTE 8-A - MAÍZ TARDÍO DK692 (V8)
Superficie: 250 hectáreas
Fecha de siembra: 05/12/2023

MONITOREO DE PLAGAS:
1. COGOLLERO (Spodoptera frugiperda):
   - Infestación: 45% de plantas con presencia
   - Severidad: 25% plantas con daño >20% foliar
   - Umbral: 20% plantas con daño
   - Recomendación: Aplicación urgente con Bt o spinosinas

2. BARRENADOR DEL TALLO (Diatraea saccharalis):
   - Presencia: Detectada en borduras
   - Nivel: Por debajo del umbral económico
   - Monitoreo preventivo requerido

CONDICIONES METEOROLÓGICAS:
Temperatura máxima: 28°C
Humedad relativa: 75%
Precipitaciones última semana: 15 mm
"""

# System prompt especializado en análisis fitosanitario
system_prompt_fitosanitario = """
Eres un especialista en protección vegetal con expertise en manejo integrado de plagas (MIP).
Procesas reportes fitosanitarios para generar datos estructurados y recomendaciones técnicas.
Tu análisis incluye:
- Identificación de plagas y niveles poblacionales
- Comparación con umbrales de daño económico
- Priorización de tratamientos según criterios técnico-económicos
- Recomendaciones de productos y dosis según buenas prácticas

Devuelve información en formato JSON estructurado para posterior análisis.
"""

# Prompt para extracción de datos fitosanitarios
prompt_fitosanitario = f"""
Analiza el siguiente reporte fitosanitario y extrae la información en formato JSON.
Estructura requerida:
{{
    "fecha_reporte": "string",
    "responsable": "string",
    "lotes": [
        {{
            "identificador": "string",
            "cultivo": "string",
            "superficie_ha": float,
            "fecha_siembra": "string",
            "estado_fenologico": "string",
            "plagas": [
                {{
                    "nombre_comun": "string",
                    "nombre_cientifico": "string",
                    "densidad_observada": float,
                    "unidad_medida": "string",
                    "umbral_economico": float,
                    "superado_umbral": boolean,
                    "recomendacion": "string",
                    "prioridad": "string"
                }}
            ]
        }}
    ],
    "condiciones_meteorologicas": {{
        "temperatura_max": float,
        "humedad_relativa": float,
        "precipitaciones_mm": float
    }}
}}

REPORTE A PROCESAR:
{reporte_fitosanitario}

Devuelve SOLO el JSON válido, sin texto adicional.
"""

# Configurar mensajes
mensajes_fitosanitarios = [
    {"role": "system", "content": system_prompt_fitosanitario},
    {"role": "user", "content": prompt_fitosanitario}
]

print("Procesando reporte fitosanitario...")
print("Analizando umbrales de daño económico y recomendaciones...")

# Si el archivo no existe, creamos el reporte de ejemplo
import os
archivo_fitosanitario = "reporte_fitosanitario.txt"
if not os.path.exists(archivo_fitosanitario):
    with open(archivo_fitosanitario, "w", encoding="utf-8") as f:
        f.write(reporte_fitosanitario)

# Procesamiento con IA local
response = chat(model=MODEL_NAME, messages=mensajes_fitosanitarios)

try:
    texto_respuesta = response.message.content
except Exception:
    texto_respuesta = str(response)

print('Respuesta del modelo:')
print(texto_respuesta[:500] + "..." if len(texto_respuesta) > 500 else texto_respuesta)

# Extraer JSON y crear análisis
def extract_json_array(text: str):
    start = text.find('{')
    end = text.rfind('}')
    if start != -1 and end != -1 and end > start:
        candidate = text[start:end+1]
        try:
            return json.loads(candidate)
        except Exception:
            return None
    return None

datos_fitosanitarios = extract_json_array(texto_respuesta)
if not datos_fitosanitarios:
    print('\nError en extracción JSON. Mostrando respuesta original.')
else:
    print('\n' + "="*70)
    print('ANÁLISIS FITOSANITARIO ESTRUCTURADO:')
    print("="*70)

    # Crear DataFrame consolidado de plagas
    plagas_consolidadas = []
    for lote in datos_fitosanitarios['lotes']:
        for plaga in lote['plagas']:
            plagas_consolidadas.append({
                'lote': lote['identificador'],
                'cultivo': lote['cultivo'],
                'superficie_ha': lote['superficie_ha'],
                'plaga': plaga['nombre_comun'],
                'densidad': plaga['densidad_observada'],
                'umbral': plaga['umbral_economico'],
                'requiere_tratamiento': plaga['superado_umbral'],
                'prioridad': plaga['prioridad']
            })

    df_plagas = pd.DataFrame(plagas_consolidadas)
    print('\nRESUMEN DE MONITOREO:')
    print(df_plagas.to_string(index=False))

    # Análisis de prioridades de tratamiento
    tratamientos_requeridos = df_plagas[df_plagas['requiere_tratamiento'] == True]
    superficie_total_tratamiento = tratamientos_requeridos['superficie_ha'].sum()

    print(f'\n--- PLAN DE ACCIÓN FITOSANITARIO ---')
    print(f'Superficie que requiere tratamiento: {superficie_total_tratamiento} hectáreas')
    print(f'Número de focos de alto riesgo: {len(tratamientos_requeridos)}')

    if len(tratamientos_requeridos) > 0:
        print('\nTRATAMIENTOS PRIORITARIOS:')
        for _, fila in tratamientos_requeridos.iterrows():
            print(f'• {fila["lote"]} ({fila["cultivo"]}): {fila["plaga"]} - {fila["prioridad"]} prioridad')

    # Guardar datos para análisis posterior
    df_plagas.to_csv('analisis_fitosanitario.csv', index=False, encoding='utf-8')
    print(f'\n✓ Datos guardados en analisis_fitosanitario.csv')

## Herramienta de Procesamiento Agropecuario: Democratización del Acceso a IA

La siguiente interfaz está diseñada para **democratizar el acceso a IA en equipos agropecuarios**, permitiendo que profesionales sin conocimientos técnicos de programación puedan procesar:

- **Reportes de campo**: Monitoreos, aplicaciones, observaciones técnicas
- **Análisis de laboratorio**: Suelos, granos, forrajes, agua
- **Informes técnicos**: Asesorías, recomendaciones, planes de manejo

### Características técnicas:
- **Procesamiento local**: Sin dependencia de conectividad externa
- **Múltiples formatos**: TXT, CSV, PDF (texto plano)
- **Análisis estructurado**: Extracción automática de datos técnicos
- **Interfaz intuitiva**: Diseñada para usuarios no técnicos
- **Exportación estándar**: Resultados en formato CSV para análisis posterior

In [ ]:
import gradio as gr
from io import StringIO
import datetime

def procesar_documento_agropecuario(archivo_subido, tipo_analisis):
    """
    Procesa documentos agropecuarios utilizando IA local especializada.

    Parámetros:
    - archivo_subido: Documento a procesar (TXT, CSV)
    - tipo_analisis: Tipo de análisis a realizar
    """

    if archivo_subido is None:
        return pd.DataFrame([{"Error": "No se ha subido ningún archivo"}]), None

    try:
        # 1. Leer el contenido del archivo
        with open(archivo_subido.name, 'r', encoding='utf-8') as f:
            contenido_documento = f.read()

        # 2. Configurar el system prompt según el tipo de análisis
        system_prompts = {
            "Reporte de Campo": """
            Eres un especialista en análisis de reportes de campo agropecuarios.
            Extraes información estructurada sobre: lotes, cultivos, fechas, observaciones,
            aplicaciones, condiciones meteorológicas y recomendaciones técnicas.
            """,
            "Análisis de Laboratorio": """
            Eres un especialista en interpretación de análisis de laboratorio agropecuario.
            Procesas resultados de: análisis de suelos, calidad de granos, forrajes,
            agua de riego y parámetros nutricionales.
            """,
            "Informe Técnico": """
            Eres un consultor agronómico especializado en informes técnicos.
            Analizas recomendaciones de manejo, planes de fertilización,
            estrategias de control sanitario y evaluaciones económicas.
            """,
            "Monitoreo Fitosanitario": """
            Eres un especialista en protección vegetal y manejo integrado de plagas.
            Procesas monitoreos de plagas, enfermedades, malezas,
            umbrales económicos y recomendaciones de tratamiento.
            """
        }

        # 3. Configurar el prompt específico
        system_prompt = system_prompts.get(tipo_analisis, system_prompts["Reporte de Campo"])

        user_prompt = f"""
        Analiza el siguiente documento agropecuario y extrae la información en formato CSV.

        IMPORTANTE:
        - Primera fila: nombres de columnas separados por comas
        - Filas siguientes: datos correspondientes a cada registro
        - Usar punto y coma (;) si hay comas en los datos
        - Incluir todas las variables técnicas relevantes
        - Mantener precisión en valores numéricos

        DOCUMENTO A PROCESAR:
        {contenido_documento}

        Devuelve ÚNICAMENTE el contenido CSV, sin explicaciones adicionales.
        """

        # 4. Llamar al modelo Ollama
        respuesta = chat(
            model=MODEL_NAME,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ]
        )

        csv_resultado = respuesta.message.content

        # 5. Procesar y validar el resultado CSV
        try:
            # Intentar leer el CSV generado
            df_resultado = pd.read_csv(StringIO(csv_resultado))

            # Agregar metadatos del procesamiento
            timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
            nombre_archivo = f"procesamiento_agropecuario_{timestamp}.csv"

            # 6. Guardar archivo de salida
            with open(nombre_archivo, 'w', encoding='utf-8') as f:
                f.write(csv_resultado)

            # 7. Crear reporte de procesamiento
            info_procesamiento = pd.DataFrame([{
                "Archivo procesado": archivo_subido.name,
                "Tipo de análisis": tipo_analisis,
                "Registros extraídos": len(df_resultado),
                "Columnas identificadas": len(df_resultado.columns),
                "Fecha de procesamiento": datetime.datetime.now().strftime("%d/%m/%Y %H:%M")
            }])

            return df_resultado, nombre_archivo, info_procesamiento

        except Exception as e:
            # Si falla el parsing CSV, mostrar el resultado crudo
            df_error = pd.DataFrame([{
                "Resultado del procesamiento": csv_resultado[:1000] + "..." if len(csv_resultado) > 1000 else csv_resultado
            }])
            return df_error, None, pd.DataFrame([{"Error": f"Error en formato CSV: {str(e)}"}])

    except Exception as e:
        error_df = pd.DataFrame([{"Error": f"Error en procesamiento: {str(e)}"}])
        return error_df, None, error_df

# Crear interfaz profesional para equipos agropecuarios
with gr.Blocks(
    title="Procesador Agropecuario con IA Local",
    theme=gr.themes.Soft()
) as interfaz_agropecuaria:

    gr.Markdown("""
    # 🌾 Procesador de Documentos Agropecuarios
    ### Análisis Inteligente con IA Local para Equipos Técnicos

    Esta herramienta permite procesar automáticamente documentos técnicos agropecuarios
    utilizando inteligencia artificial ejecutada localmente, garantizando la **confidencialidad**
    de datos sensibles del establecimiento.
    """)

    with gr.Row():
        with gr.Column(scale=1):
            archivo_entrada = gr.File(
                label="📄 Subir Documento",
                file_types=[".txt", ".csv"],
                file_count="single"
            )

            tipo_procesamiento = gr.Dropdown(
                choices=[
                    "Reporte de Campo",
                    "Análisis de Laboratorio",
                    "Informe Técnico",
                    "Monitoreo Fitosanitario"
                ],
                label="🔬 Tipo de Análisis",
                value="Reporte de Campo"
            )

            btn_procesar = gr.Button(
                "⚡ Procesar con IA Local",
                variant="primary",
                size="lg"
            )

        with gr.Column(scale=2):
            info_procesamiento = gr.Dataframe(
                label="ℹ️ Información del Procesamiento",
                interactive=False
            )

    with gr.Row():
        vista_previa = gr.Dataframe(
            label="📊 Vista Previa de Datos Extraídos",
            interactive=False,
            wrap=True
        )

    with gr.Row():
        archivo_descarga = gr.File(
            label="💾 Descargar Resultado en CSV",
            interactive=False
        )

    # Configurar la acción del botón
    btn_procesar.click(
        fn=procesar_documento_agropecuario,
        inputs=[archivo_entrada, tipo_procesamiento],
        outputs=[vista_previa, archivo_descarga, info_procesamiento]
    )

    gr.Markdown("""
    ---
    **🔒 Características de Seguridad:**
    - Procesamiento 100% local, sin envío de datos a servidores externos
    - Confidencialidad garantizada para información sensible del establecimiento
    - Compatible con normativas de protección de datos agropecuarios

    **📈 Casos de Uso Típicos:**
    - Digitalización de reportes de monitoreo de campo
    - Procesamiento de resultados de análisis de laboratorio
    - Extracción de datos de informes técnicos de asesores
    - Sistematización de registros fitosanitarios
    """)

# Ejecutar la interfaz
print("Iniciando herramienta de procesamiento agropecuario...")
print("Democratizando el acceso a IA para equipos técnicos del sector...")

# Lanzar interfaz (en producción usar share=False por seguridad)
interfaz_agropecuaria.launch(
    server_name="127.0.0.1",  # Solo acceso local por seguridad
    server_port=7860,
    share=False,  # Sin compartir públicamente
    quiet=False
)